<a href="https://colab.research.google.com/github/palVikram/Advance_tensorflow/blob/main/sentence_similarity_bert_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow-gpu==2.3.0
!pip install tensorflow transformers tf2onnx onnxruntime
!pip install transformers
!pip install numpy==1.19.5

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/base_command.py", line 180, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/req_command.py", line 199, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py", line 319, in run
    reqs, check_supported_wheels=not options.target_dir
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 128, in resolve
    requirements, max_rounds=try_to_avoid_resolution_too_deep
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 473, in resolve
    state = resolution.resolve(requirements, max_rounds=max_rounds)
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 367, in resolve
    failure_causes = self._attempt_

In [ ]:
%%bash
wget https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64/nvidia-machine-learning-repo-ubuntu1804_1.0.0-1_amd64.deb

wget https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64/nv-tensorrt-repo-ubuntu1804-cuda11.3-trt8.0.1.6-ga-20210626_1-1_amd64.deb

dpkg -i nvidia-machine-learning-repo-*.deb
apt-get update

sudo apt-get install libnvinfer5

(Reading database ... 155302 files and directories currently installed.)
Preparing to unpack nvidia-machine-learning-repo-ubuntu1804_1.0.0-1_amd64.deb ...
Unpacking nvidia-machine-learning-repo-ubuntu1804 (1.0.0-1) over (1.0.0-1) ...
Setting up nvidia-machine-learning-repo-ubuntu1804 (1.0.0-1) ...
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
libnvinf

--2022-05-08 23:33:56--  https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64/nvidia-machine-learning-repo-ubuntu1804_1.0.0-1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2926 (2.9K) [application/x-deb]
Saving to: ‘nvidia-machine-learning-repo-ubuntu1804_1.0.0-1_amd64.deb.1’

     0K ..                                                    100% 76.0M=0s

2022-05-08 23:33:56 (76.0 MB/s) - ‘nvidia-machine-learning-repo-ubuntu1804_1.0.0-1_amd64.deb.1’ saved [2926/2926]

--2022-05-08 23:33:56--  https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64/nv-tensorrt-repo-ubuntu1804-cuda11.3-trt8.0.1.6-ga-20210626_1-1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
C

In [ ]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
gpu = gpus[0]

tf.config.experimental.set_memory_growth(gpu, True)
print("Tensorflow version: ", tf.version.VERSION)

Tensorflow version:  2.3.0


In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
!curl -LO https://raw.githubusercontent.com/MohamadMerchant/SNLI/master/data.tar.gz
!tar -xvzf data.tar.gz
# There are more than 550k samples in total; we will use 100k for this example.
train_df = pd.read_csv("SNLI_Corpus/snli_1.0_train.csv", nrows=5000)
valid_df = pd.read_csv("SNLI_Corpus/snli_1.0_dev.csv")
test_df = pd.read_csv("SNLI_Corpus/snli_1.0_test.csv")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11.1M  100 11.1M    0     0  75.0M      0 --:--:-- --:--:-- --:--:-- 75.0M
SNLI_Corpus/
SNLI_Corpus/snli_1.0_dev.csv
SNLI_Corpus/snli_1.0_train.csv
SNLI_Corpus/snli_1.0_test.csv


In [ ]:
train_df.head()

,similarity,sentence1,sentence2
0,neutral,A person on a horse jumps over a broken down a...,A person is training his horse for a competition.
1,contradiction,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette."
2,entailment,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse."
3,neutral,Children smiling and waving at camera,They are smiling at their parents
4,entailment,Children smiling and waving at camera,There are children present


In [ ]:
train_df['is_duplicate']=1
train_df['is_duplicate'][train_df['similarity']=='entailment']=1
train_df['is_duplicate'][train_df['similarity']=='contradiction']=0

In [ ]:
train_df=train_df[(train_df['similarity']=='entailment') | (train_df['similarity']=='contradiction')]

In [ ]:
train_df['question1']=train_df['sentence1']
train_df['question2']=train_df['sentence2']

train_df.drop(['similarity','sentence1','sentence2'], axis=1, inplace=True)
train_df=train_df.reset_index()

In [ ]:
train_df.drop('index',axis=1, inplace=True)

In [ ]:
train_df.head()

,is_duplicate,question1,question2
0,0,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette."
1,1,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse."
2,1,Children smiling and waving at camera,There are children present
3,0,Children smiling and waving at camera,The kids are frowning
4,0,A boy is jumping on skateboard in the middle o...,The boy skates down the sidewalk.


In [ ]:
['my name is vikram']*10

['my name is vikram',
 'my name is vikram',
 'my name is vikram',
 'my name is vikram',
 'my name is vikram',
 'my name is vikram',
 'my name is vikram',
 'my name is vikram',
 'my name is vikram',
 'my name is vikram']

In [ ]:
valid_df['is_duplicate']=1
valid_df['is_duplicate'][valid_df['similarity']=='entailment']=1
valid_df['is_duplicate'][valid_df['similarity']=='contradiction']=0

valid_df=valid_df[(valid_df['similarity']=='entailment') | (valid_df['similarity']=='contradiction')]

test_df['is_duplicate']=1
test_df['is_duplicate'][test_df['similarity']=='entailment']=1
test_df['is_duplicate'][test_df['similarity']=='contradiction']=0

test_df=test_df[(test_df['similarity']=='entailment') | (test_df['similarity']=='contradiction')]

In [ ]:
valid_df['question1']=valid_df['sentence1']
valid_df['question2']=valid_df['sentence2']

valid_df.drop(['similarity','sentence1','sentence2'], axis=1, inplace=True)

valid_df=valid_df.reset_index()

valid_df.drop('index',axis=1, inplace=True)

In [ ]:
test_df['question1']=test_df['sentence1']
test_df['question2']=test_df['sentence2']

test_df.drop(['similarity','sentence1','sentence2'], axis=1, inplace=True)

test_df=test_df.reset_index()

test_df.drop('index',axis=1, inplace=True)

In [ ]:
with open('paraphrased_sentences.txt') as f:
    paraphrased_data = f.read().splitlines()

paraphrased_data=[data for data in paraphrased_data if len(data)>0]

In [ ]:
with open('training_data.txt') as f:
    cleaned_data = f.read().splitlines()

cleaned=[]
for data in cleaned_data:
      if '(' in data or ')' in data:
        data1=(data[:data.index('(')]+data[data.index(')')+1:]).strip()
        data1=data1.replace("[","").replace("]","")
        if '(' in data1 or ')' in data1:
          data1=(data1[:data1.index('(')]+data1[data1.index(')')+1:]).strip()
          data1=data1.replace("[","").replace("]","")
        cleaned.append(data1)
      else: 
        data=data.replace("[","").replace("]","").strip()
        cleaned.append(data)
len(cleaned)

471

In [ ]:
left_sentence=[]
right_sentence=[]

for left, right in zip(cleaned, paraphrased_data):
  if len(left)>1 and len(right)>1:
    left_sentence.append(left)
    right_sentence.append(right)

In [ ]:
left_sentence=cleaned
right_sentence=paraphrased_data
len(left_sentence),len(right_sentence)

df_data=pd.DataFrame(list(zip(left_sentence,right_sentence)), columns=["question1", "question2"])

df_data["is_duplicate"]=1
df_data

,question1,question2,is_duplicate
0,can you give me news about finance,Can you provide me with financial information?,1
1,can you tell me the news,Could you please inform me of the latest devel...,1
2,give me today's headlines,Please send me today's news.,1
3,give me today's news,Please provide me with today's news.,1
4,play the news,turn on the news,1
...,...,...,...
461,what are npr financial channels,what content do you have,1
462,what are shows I could watch,what else can you do,1
463,what are the top programs,what is available for listening,1
464,what are the top talk shows,what shows can you play,1


In [ ]:
import numpy as np
###df_data["label"]=np.where(df_data["label"].index<465, df_data["label"], 0)

df_merge=pd.concat([df_data,train_df], axis=0).reset_index()

df_merge

,index,question1,question2,is_duplicate
0,0,can you give me news about finance,Can you provide me with financial information?,1
1,1,can you tell me the news,Could you please inform me of the latest devel...,1
2,2,give me today's headlines,Please send me today's news.,1
3,3,give me today's news,Please provide me with today's news.,1
4,4,play the news,turn on the news,1
...,...,...,...,...
3799,3333,A man standing on a motorcycle doing a stunt,a man crashes a car into a pole,0
3800,3334,Motorcycle daredevil on reared motorcycle.,Motorcyle crashes into a pile of cars.,0
3801,3335,Motorcycle daredevil on reared motorcycle.,Motorcylist rides his bike.,1
3802,3336,Someone riding sideways on a motorcycle.,The rider is straddling the motorcycle.,0


In [ ]:
df_merge.drop('index',axis=1, inplace=True)

In [ ]:
df_merge=df_merge.dropna()
df_merge.isna().sum()

question1       0
question2       0
is_duplicate    0
dtype: int64

In [ ]:
#df_merge['question1']=df_merge['question1'].apply(lambda x: remove_stopwords(x))

In [ ]:
df_merge['is_duplicate'].value_counts()

1    2137
0    1667
Name: is_duplicate, dtype: int64

In [ ]:
df_merge['question1'].tail(30)

3774    A man hovers over a woman seated at an art gal...
3775                            Racedog has the number 8.
3776                            Racedog has the number 8.
3777        A dog, wearing a number 8, running in a race.
3778        A dog, wearing a number 8, running in a race.
3779    A greyhound runs in a race, bee striped jacket...
3780    A greyhound runs in a race, bee striped jacket...
3781    A greyhound dog wearing a yellow and black shi...
3782    A greyhound dog wearing a yellow and black shi...
3783    A greyhound dog wearing a yellow and black shi...
3784    A greyhound dog wearing a yellow and black shi...
3785    A greyhound dog wearing a yellow and black shi...
3786    A greyhound dog wearing a yellow and black shi...
3787    A greyhound dog wearing a yellow and black shi...
3788    A greyhound dog wearing a yellow and black shi...
3789    A greyhound dog wearing a yellow and black shi...
3790    A greyhound dog wearing a yellow and black shi...
3791    Gray d

In [ ]:
#df_merge['question2']=df_merge['question2'].apply(lambda x: remove_stopwords(x))

In [ ]:
left_train_x=df_merge['question1'].tolist()
right_train_x=df_merge['question2'].tolist()

In [ ]:
#valid_df['question1']=valid_df['question1'].apply(lambda x: remove_stopwords(x))
#valid_df['question2']=valid_df['question2'].apply(lambda x: remove_stopwords(x))

In [ ]:
#test_df['question1']=test_df['question1'].apply(lambda x: remove_stopwords(x))
#test_df['question2']=test_df['question2'].apply(lambda x: remove_stopwords(x))

In [ ]:
question1_list=test_df['question1'].tolist()
question2_list=test_df['question2'].tolist()
y_list=test_df['is_duplicate'].tolist()

In [ ]:
left_test_x=valid_df['question1'].tolist()
right_test_x=valid_df['question2'].tolist()

In [ ]:
import transformers

In [ ]:
max_length = 128  # Maximum length of input sentence to the model.
batch_size = 16
epochs = 2

# Labels in our dataset.
labels = ["contradiction", "entailment"]

In [ ]:
tokenizer = transformers.BertTokenizer.from_pretrained("huawei-noah/TinyBERT_4L_zh", do_lower_case=True)
#tokenizer = transformers.GPT2Tokenizer.from_pretrained("gpt2", do_lower_case=True)

In [ ]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

0

In [ ]:
class BertSemanticDataGenerator(tf.keras.utils.Sequence):
    """Generates batches of data.

    Args:
        sentence_pairs: Array of premise and hypothesis input sentences.
        labels: Array of labels.
        batch_size: Integer batch size.
        shuffle: boolean, whether to shuffle the data.
        include_targets: boolean, whether to incude the labels.

    Returns:
        Tuples `([input_ids, attention_mask, `token_type_ids], labels)`
        (or just `[input_ids, attention_mask, `token_type_ids]`
         if `include_targets=False`)
    """

    def __init__(
        self,
        sentence_pairs,
        labels,
        batch_size=batch_size,
        shuffle=True,
        include_targets=True,
    ):
        self.sentence_pairs = sentence_pairs
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        # Load our BERT Tokenizer to encode the text.
        # We will use base-base-uncased pretrained model.
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()

    def __len__(self):
        # Denotes the number of batches per epoch.
        return len(self.sentence_pairs) // self.batch_size

    def __getitem__(self, idx):
        # Retrieves the batch of index.
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence_pairs = self.sentence_pairs[indexes]

        # With BERT tokenizer's batch_encode_plus batch of both the sentences are
        # encoded together and separated by [SEP] token.
        encoded = tokenizer.batch_encode_plus(
            sentence_pairs.tolist(),
            add_special_tokens=True,
            max_length=max_length,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            return_tensors="tf",
        )

        # Convert batch of encoded features to numpy array.
        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        # Set to true if data generator is used for training/validation.
        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

    def on_epoch_end(self):
        # Shuffle indexes after each epoch if shuffle is set to True.
        if self.shuffle:
            np.random.RandomState(42).shuffle(self.indexes)

In [ ]:
# Create the model under a distribution strategy scope.
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    # Encoded token ids from BERT tokenizer.
    input_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="input_ids"
    )
    # Attention masks indicates to the model which tokens should be attended to.
    attention_masks = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="attention_masks"
    )
    # Token type ids are binary masks identifying different sequences in the model.
    token_type_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="token_type_ids"
    )
    # Loading pretrained BERT model.
    bert_model = transformers.TFBertModel.from_pretrained("huawei-noah/TinyBERT_4L_zh",from_pt=True)
    #bert_model = transformers.GPT2Model.from_pretrained("gpt2")
    # Freeze the BERT model to reuse the pretrained features without modifying them.
    bert_model.trainable = False

    bert_output = bert_model(
        input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids
    )
    sequence_output = bert_output.last_hidden_state
    #pooled_output = bert_output.pooler_output
    # Add trainable layers on top of frozen layers to adapt the pretrained features on the new data.
    bi_lstm = tf.keras.layers.Bidirectional(
        
        tf.keras.layers.LSTM(64, return_sequences=True)
    )(sequence_output)
    
    # Applying hybrid pooling approach to bi_lstm sequence output.
    avg_pool = tf.keras.layers.GlobalAveragePooling1D()(bi_lstm)
    max_pool = tf.keras.layers.GlobalMaxPooling1D()(bi_lstm)
    concat = tf.keras.layers.concatenate([avg_pool, max_pool])
    dropout = tf.keras.layers.Dropout(0.3)(concat)
    output = tf.keras.layers.Dense(1, activation="sigmoid")(dropout)
    model = tf.keras.models.Model(
        inputs=[input_ids, attention_masks, token_type_ids], outputs=output
    )

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss="binary_crossentropy",
        metrics=["acc"],
    )


print(f"Strategy: {strategy}")
model.summary()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'fit_dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'fit_dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the chec

Strategy: <tensorflow.python.distribute.mirrored_strategy.MirroredStrategy object at 0x7fd8d688f4d0>
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 128)]        0                                            
__________________________________________________________________________________________________
attention_masks (InputLayer)    [(None, 128)]        0                                            
__________________________________________________________________________________________________
token_type_ids (InputLayer)     [(None, 128)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     TFBaseModelOutputWit 11419320    input_ids[0][0]     

In [ ]:
train_df['is_duplicate'].value_counts()

1    1671
0    1667
Name: is_duplicate, dtype: int64

In [ ]:
train_data = BertSemanticDataGenerator(
    train_df[["question1", "question2"]].values.astype("str"),
    train_df['is_duplicate'],
    batch_size=batch_size,
    shuffle=True,
)

valid_data = BertSemanticDataGenerator(
    valid_df[["question1", "question2"]].values.astype("str"),
    valid_df['is_duplicate'],
    batch_size=batch_size,
    shuffle=False,
)

In [ ]:
history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=epochs,
    use_multiprocessing=True,
    workers=-1,
)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Epoch 1/2
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


208/208 [==============================] - ETA: 0s - loss: 0.7118 - acc: 0.5337INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

208/208 [==============================] - 19s 93ms/step - loss: 0.7118 - acc: 0.5337 - val_loss: 0.6688 - val_acc: 0.5237
Epoch 2/2
 79/208 [==========>...................] - ETA: 3s - loss: 0.6693 - acc: 0.5854

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


208/208 [==============================] - ETA: 0s - loss: 0.6562 - acc: 0.6076

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

208/208 [==============================] - 16s 79ms/step - loss: 0.6562 - acc: 0.6076 - val_loss: 0.5960 - val_acc: 0.7386


In [ ]:
# Unfreeze the bert_model.
bert_model.trainable = True
# Recompile the model to make the change effective.
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="binary_crossentropy",

    metrics=["accuracy"],
)
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 128)]        0                                            
__________________________________________________________________________________________________
attention_masks (InputLayer)    [(None, 128)]        0                                            
__________________________________________________________________________________________________
token_type_ids (InputLayer)     [(None, 128)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     TFBaseModelOutputWit 11419320    input_ids[0][0]                  
                                                                 attention_masks[0][0] 

In [ ]:
history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=epochs,
    use_multiprocessing=True,
    workers=-1,
)

In [ ]:
test_data = BertSemanticDataGenerator(
    test_df[["question1", "question2"]].values.astype("str"),
    test_df['is_duplicate'],
    batch_size=batch_size,
    shuffle=False,
)
model.evaluate(test_data, verbose=1)

 91/412 [=====>........................] - ETA: 46s - loss: 0.1224 - accuracy: 0.9533

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


 92/412 [=====>........................] - ETA: 45s - loss: 0.1254 - accuracy: 0.9531

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


109/412 [======>.......................] - ETA: 43s - loss: 0.1245 - accuracy: 0.9541

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


412/412 [==============================] - 60s 145ms/step - loss: 0.1213 - accuracy: 0.9539


[0.12133292853832245, 0.9538834691047668]

In [ ]:
def check_similarity(sentence1, sentence2):
    sentence_pairs = np.array([[str(sentence1), str(sentence2)]])
    test_data = BertSemanticDataGenerator(
        sentence_pairs, labels=None, batch_size=1, shuffle=False, include_targets=False,
    )

    proba = model.predict(test_data[0])[0]
    print(proba)
    idx = np.argmax(proba)
    
    proba = f"{proba[idx]: .2f}%"
    return proba

In [ ]:
cleaned=[clean for clean in cleaned if len(clean)>1]

In [ ]:
input_infer=['play npr news']*len(cleaned)

In [ ]:
final_result=[]
encoded_pair=[]
for sentence1, sentence2 in zip(input_infer[:400], cleaned[:400]):
    sentence_pairs = np.array([[str(sentence1), str(sentence2)]])
    test_data = BertSemanticDataGenerator(
        sentence_pairs, labels=None, batch_size=1, shuffle=False, include_targets=False,
    )
    encoded_pair.append(test_data)

for test_data in encoded_pair:
    proba = model.predict(test_data[0])[0]  
    final_result.append(proba)

final_result

[array([0.00250908], dtype=float32),
 array([0.89061344], dtype=float32),
 array([0.6832818], dtype=float32),
 array([0.7680873], dtype=float32),
 array([0.9351207], dtype=float32),
 array([0.93466645], dtype=float32),
 array([0.00244383], dtype=float32),
 array([0.6777355], dtype=float32),
 array([0.97911924], dtype=float32),
 array([0.9587685], dtype=float32),
 array([0.98704696], dtype=float32),
 array([0.9587685], dtype=float32),
 array([0.88675094], dtype=float32),
 array([0.08993897], dtype=float32),
 array([0.9178036], dtype=float32),
 array([0.9821231], dtype=float32),
 array([0.726428], dtype=float32),
 array([0.6865537], dtype=float32),
 array([0.099333], dtype=float32),
 array([0.82516855], dtype=float32),
 array([0.88854825], dtype=float32),
 array([0.4001964], dtype=float32),
 array([0.81479377], dtype=float32),
 array([0.8539466], dtype=float32),
 array([0.9821231], dtype=float32),
 array([0.81739545], dtype=float32),
 array([0.05912878], dtype=float32),
 array([0.0324760

In [ ]:
#train_df[["question1", "question2"]].values[:450]

In [ ]:
final_result=[]
for sentence1, sentence2 in zip(input_infer, cleaned):
    sentence_pairs = np.array([[str(sentence1), str(sentence2)]])
    test_data = BertSemanticDataGenerator(
        sentence_pairs, labels=None, batch_size=1, shuffle=False, include_targets=False,
    )
    proba = model.predict(test_data[0])[0]  
    final_result.append(proba)

final_result

In [ ]:
final_result=[]
for sentence1, sentence2 in zip(input_infer, cleaned):
    sentence_pairs = np.array([[str(sentence1), str(sentence2)]])
    test_data = BertSemanticDataGenerator(
        sentence_pairs, labels=None, batch_size=1, shuffle=False, include_targets=False,
    )
    proba = model.predict(test_data[0])[0]  
    final_result.append(proba)

final_result

In [ ]:
!pip install numpy --upgrade

  Using cached numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-gpu 2.3.0 requires numpy<1.19.0,>=1.16.0, but you have numpy 1.21.6 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
import numpy as np
import onnxruntime as rt
import tensorflow as tf
import tf2onnx

In [ ]:
# describe the inputs
input_spec = (
    tf.TensorSpec((1,max_length,), tf.int32, name="input_ids"),
    tf.TensorSpec((1,max_length,), tf.int32, name="attention_masks"),
    tf.TensorSpec((1,max_length), tf.int32, name="token_type_ids")
)

In [ ]:
sentence1 = "play news"
sentence2 = "play news"

sentence_pairs = np.array([[str(sentence1), str(sentence2)]])
input_dict = BertSemanticDataGenerator(
    sentence_pairs, labels=None, batch_size=1, shuffle=False, include_targets=False,
)

tf_results = model.predict(input_dict[0])[0]
tf_results

array([0.9696498], dtype=float32)

In [ ]:
# and convert
_, _ = tf2onnx.convert.from_keras(model, input_signature=input_spec, opset=13, output_path="bert.onnx")

Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


rnn LSTM__63 following Reverse op isn't the part of bi-rnn.


In [ ]:
opt = rt.SessionOptions()
sess = rt.InferenceSession("bert.onnx", None)

input_ids=sess.get_inputs()[0].name
attention_mask=sess.get_inputs()[1].name
token_type_ids=sess.get_inputs()[2].name

In [ ]:
input_dict_np={input_ids:input_dict[0][0],attention_mask:input_dict[0][1], token_type_ids:input_dict[0][2]}
input_dict_np
onnx_results = sess.run(None,input_dict_np)

In [ ]:
final_result=[]
encoded_pair=[]
for sentence1, sentence2 in zip(input_infer[:400], cleaned[:400]):
    sentence_pairs = np.array([[str(sentence1), str(sentence2)]])
    test_data = BertSemanticDataGenerator(
        sentence_pairs, labels=None, batch_size=1, shuffle=False, include_targets=False,
    )
    encoded_pair.append(test_data)

In [ ]:
for input_dict in encoded_pair:
    input_dict_np={input_ids:input_dict[0][0],attention_mask:input_dict[0][1], token_type_ids:input_dict[0][2]}
    onnx_results = sess.run(None,input_dict_np)
    final_result.append(onnx_results)

final_result

[[array([[0.00250906]], dtype=float32)],
 [array([[0.89061344]], dtype=float32)],
 [array([[0.68328196]], dtype=float32)],
 [array([[0.7680876]], dtype=float32)],
 [array([[0.9351208]], dtype=float32)],
 [array([[0.93466645]], dtype=float32)],
 [array([[0.00244379]], dtype=float32)],
 [array([[0.6777359]], dtype=float32)],
 [array([[0.9791193]], dtype=float32)],
 [array([[0.95876855]], dtype=float32)],
 [array([[0.98704696]], dtype=float32)],
 [array([[0.95876855]], dtype=float32)],
 [array([[0.88675094]], dtype=float32)],
 [array([[0.08993891]], dtype=float32)],
 [array([[0.91780365]], dtype=float32)],
 [array([[0.98212314]], dtype=float32)],
 [array([[0.7264276]], dtype=float32)],
 [array([[0.6865536]], dtype=float32)],
 [array([[0.09933332]], dtype=float32)],
 [array([[0.8251691]], dtype=float32)],
 [array([[0.888548]], dtype=float32)],
 [array([[0.40019608]], dtype=float32)],
 [array([[0.8147938]], dtype=float32)],
 [array([[0.8539467]], dtype=float32)],
 [array([[0.98212314]], dty

In [ ]:
import onnxruntime as ort

providers = [
    ('TensorrtExecutionProvider', {
        'device_id': 1,
        'trt_max_workspace_size': 2147483648,
        'trt_fp16_enable': True,
    }),
    ('CUDAExecutionProvider', {
        'device_id': 1,
        'arena_extend_strategy': 'kNextPowerOfTwo',
        'gpu_mem_limit': 2 * 1024 * 1024 * 1024,
        'cudnn_conv_algo_search': 'EXHAUSTIVE',
        'do_copy_in_default_stream': True,
    })
]

sess_opt = ort.SessionOptions()
sess = ort.InferenceSession('bert.onnx', sess_options=sess_opt, providers=providers)

In [ ]:
for input_dict in encoded_pair:
    input_dict_np={input_ids:input_dict[0][0],attention_mask:input_dict[0][1], token_type_ids:input_dict[0][2]}
    onnx_results = sess.run(None,input_dict_np)
    final_result.append(onnx_results)

final_result